In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['Seller_Type'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df['current_year']=2020
df['No_of_years']=df['current_year']-df['Year']
df.head()
df=df.drop(['current_year'],axis=1)

In [ ]:
df=df.drop(['Year'],axis=1)

In [ ]:
df=df.drop(['Car_Name'],axis=1)

In [ ]:
df=pd.get_dummies(df,drop_first=True)

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
sns.pairplot(df)


In [ ]:
sns.heatmap(df.corr(),annot=True)

Selling price and Present price highly positively corelated. whereas Fuel_Type_Diesel and Fuel_Type_Petrol are less corelated. If my dataset has more number of columns, I will drop columns which are corelated.

In [ ]:
X=df.iloc[:,1:]

In [ ]:
y=df.iloc[:,:1]

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
ext=ExtraTreesRegressor()
ext.fit(X,y)

In [ ]:
ext.feature_importances_

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(X_train,y_train)

In [ ]:
prediction=model.predict(X_test)

In [ ]:
prediction

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(prediction,y_test)

As we got 0.79 accuracy score in test dataset. We should improve it. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
random_model=RandomizedSearchCV(estimator=model,param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
random_model.fit(X_train,y_train)

In [ ]:
random_model.best_params_

In [ ]:
predictions_hp=random_model.predict(X_test)

In [ ]:
mean_absolute_error(predictions_hp,y_test)

Accuracy was an increase to 0.875. 

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test,predictions_hp)